In [7]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [51]:
facebook_web_log = pd.read_excel('./Time Between Two Events Data.xlsx')

In [104]:
df = facebook_web_log[(facebook_web_log['action'] == 'page_load')
                      |(facebook_web_log['action'] == 'scroll_down')]
df.sort_values(by = ['user_id','timestamp'], inplace = True)
df['event_id'] = df.index
df = df[['event_id','user_id','timestamp','action']]

In [67]:
last_load_event = -1
last_user =  -1
def l_load(row):
    global last_load_event
    global last_user
    if (row['action'] == 'page_load'):
        last_load_event = row['event_id']
        row['prev_load_event'] = last_load_event
    elif row['action'] == 'scroll_down':
        row['prev_load_event'] = last_load_event
    return row['prev_load_event']


In [70]:
df['prev_load_event'] = -1

In [72]:
df

,event_id,user_id,timestamp,action,prev_load_event
0,0,0,2019-04-25 13:30:15,page_load,-1
1,1,0,2019-04-25 13:30:18,page_load,-1
2,2,0,2019-04-25 13:30:40,scroll_down,-1
3,3,0,2019-04-25 13:31:10,scroll_down,-1
4,4,0,2019-04-25 13:31:25,scroll_down,-1
5,5,0,2019-04-28 13:30:40,scroll_down,-1
6,6,0,2019-04-28 14:30:10,page_load,-1
7,7,0,2019-04-28 14:30:15,page_load,-1
8,8,1,2019-04-25 13:40:00,page_load,-1
9,9,1,2019-04-25 13:40:10,scroll_down,-1


In [105]:
df['prev_load_event'] = df.apply(l_load,axis=1)

In [106]:
df_scroll = df[df['action'] == 'scroll_down'][['user_id','timestamp','prev_load_event']].rename(columns = {'timestamp':'ts_scroll'})

In [114]:
df_pload  = df[df['action'] == 'page_load'][['event_id','timestamp']].rename(columns = {'timestamp':'ts_load'})

In [108]:
df_pload

,event_id,user_id,timestamp
0,0,0,2019-04-25 13:30:15
1,1,0,2019-04-25 13:30:18
6,6,0,2019-04-28 14:30:10
7,7,0,2019-04-28 14:30:15
8,8,1,2019-04-25 13:40:00
14,14,1,2019-04-26 11:15:00
17,17,2,2019-04-25 13:41:21


In [115]:
df2 = df_scroll.merge(df_pload, how = 'left', left_on = 'prev_load_event' , right_on = 'event_id').drop(columns = ['prev_load_event','event_id'])

In [121]:
df2 = df2[['user_id','ts_load','ts_scroll']]

In [131]:
df2_dd  = df2.drop_duplicates(subset=['user_id','ts_load'], keep='first').copy()

In [129]:
df2_dd

,user_id,ts_load,ts_scroll
0,0,2019-04-25 13:30:18,2019-04-25 13:30:40
4,1,2019-04-25 13:40:00,2019-04-25 13:40:10
9,1,2019-04-26 11:15:00,2019-04-26 11:15:10
11,2,2019-04-25 13:41:21,2019-04-25 13:41:30


In [132]:
df2_dd['tb_in_seconds'] = (df2_dd['ts_scroll'] - df2_dd['ts_load']).dt.seconds

In [133]:
df2_dd

,user_id,ts_load,ts_scroll,tb_in_seconds
0,0,2019-04-25 13:30:18,2019-04-25 13:30:40,22
4,1,2019-04-25 13:40:00,2019-04-25 13:40:10,10
9,1,2019-04-26 11:15:00,2019-04-26 11:15:10,10
11,2,2019-04-25 13:41:21,2019-04-25 13:41:30,9


In [136]:
df2_dd.groupby(['user_id']).agg('Sum' = ('tb_in_seconds':'sum')).reset_index()

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (3341417136.py, line 1)

In [137]:
df2_dd

,user_id,ts_load,ts_scroll,tb_in_seconds
0,0,2019-04-25 13:30:18,2019-04-25 13:30:40,22
4,1,2019-04-25 13:40:00,2019-04-25 13:40:10,10
9,1,2019-04-26 11:15:00,2019-04-26 11:15:10,10
11,2,2019-04-25 13:41:21,2019-04-25 13:41:30,9


In [138]:
df2_dd['s1'] = -100
df2_dd['s2'] = -100

In [144]:
def myf(row):
    row['s1'] = row['user_id'] + row['tb_in_seconds']
    row['s2'] = row['user_id'] - row['tb_in_seconds']
    return row[['s1','s2']]


In [146]:
df2_dd[['s1','s2']]= df2_dd.apply(myf, axis =1 )

In [147]:
df2_dd

,user_id,ts_load,ts_scroll,tb_in_seconds,s1,s2
0,0,2019-04-25 13:30:18,2019-04-25 13:30:40,22,22,-22
4,1,2019-04-25 13:40:00,2019-04-25 13:40:10,10,11,-9
9,1,2019-04-26 11:15:00,2019-04-26 11:15:10,10,11,-9
11,2,2019-04-25 13:41:21,2019-04-25 13:41:30,9,11,-7
